In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import itertools
import os
from selenium.webdriver.support import expected_conditions
import json
from sc_header import createDriver, agregar_links , scrape_url, scrape_batch,aplanar_lista, check_length

In [2]:
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')
 


In [3]:
#browser.delete_all_cookies()

In [4]:
browser = createDriver()

In [5]:
browser.delete_all_cookies()

In [6]:
links_sin_color = [['https://www.shein.com/Jeans-c-1934.html?icn=jeans&ici=www_tab01navbar04menu21dir02',
                    'JEAN'],
                   ['https://www.shein.com/Denim-Jackets-c-1933.html?icn=denim-jackets&ici=www_tab01navbar04menu21dir01',
                    'CHAQUETA DE JEAN'],
                   ['https://www.shein.com/Denim-Shorts-c-1935.html?icn=denim-shorts&ici=www_tab01navbar04menu21dir03',
                   'SHORT DE JEAN'],
                   ['https://www.shein.com/Leggings-c-1871.html?icn=leggings&ici=www_tab01navbar04menu12dir02',
                   'CALZA'],
                   ['https://www.shein.com/Active-Tops-c-2181.html?icn=active-tops&ici=www_tab01navbar04menu22dir02',
                   'TOP DEPORTIVO'],
                   ['https://www.shein.com/Skirts-c-1732.html?icn=skirts&ici=www_tab01navbar04menu12dir03',
                   'FALDA'],
                   ['https://www.shein.com/Shorts-c-1912.html?icn=shorts&ici=www_tab01navbar04menu12dir04',
                   'SHORT'],
                   ['https://www.shein.com/Pants-c-1740.html?icn=pants&ici=www_tab01navbar04menu12dir01',
                   'PANTALON']
                    ]

In [7]:
url_base = 'https://www.shein.com'
Tlistas = []
for lsc in links_sin_color:
    
    agregar_links(lsc[0],
                 browser,
                 Tlistas,
                 lsc[1],
                 url_base)
    

In [8]:
set([item[0] for item in Tlistas])

{'CALZA',
 'CHAQUETA DE JEAN',
 'FALDA',
 'JEAN',
 'PANTALON',
 'SHORT',
 'SHORT DE JEAN',
 'TOP DEPORTIVO'}

In [9]:
set([item[1] for item in Tlistas])

{'Army Green',
 'Beige',
 'Black',
 'Black and White',
 'Blue',
 'Bright',
 'Brown',
 'Burgundy',
 'Camel',
 'Gold',
 'Green',
 'Grey',
 'Khaki',
 'Multicolor',
 'Navy',
 'Nude',
 'Ombre',
 'Orange',
 'Pastel',
 'Pink',
 'Purple',
 'Red',
 'White',
 'Yellow'}

In [10]:
#browser = createDriver()
hrefs_list = []
ex_col = []
for lista in Tlistas:
    try:
        browser.get(lista[2])
    except:
        browser.quit()
        time.sleep(1)
        browser = createDriver()
        browser.get(lista[2])
    
    while True:
        soup = BeautifulSoup(browser.page_source,'html.parser')
        
        indice_=0
        for element in soup.select('div.j-switch-color-wrap > div.c-goodsitem__ratiowrap > a'):
        #for element in soup.find_all(class_='c-goodsitem__ratiowrap'):
            try:
                
                href_aux = url_base +'/'+ element['href']
                
                hrefs_list.append([indice_,
                                   lista[0],
                                   lista[1],
                                   lista[2],
                                   href_aux
                                  ])
                indice_+=1
                
            except:
                ex_col.append([lista[2],lista[1]])
                pass            
        try:
            browser.find_element_by_class_name('page-next').find_element_by_tag_name('a').click()
        except:
            break
browser.quit()

In [11]:
df = pd.DataFrame(hrefs_list,columns=['pos','tipo','color','url_scraper','href'])

In [12]:
df['auxi'] = df['href'].apply(check_length)

In [13]:
len(df),len(df.drop_duplicates())

(4932, 4932)

In [14]:
if len(df['tipo'].unique().tolist()) != len(set([item[0] for item in Tlistas])):
    print('REVISAR LOS TIPOS SCRAPEADOS')
    raise

In [15]:
start_ = datetime.datetime.now()
items = []
errores = []

batch_size = 5
url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

var = 0
for ix, url_chunk in enumerate(url_chunks):
    scrape_batch(errores,items,url_chunk)
    
    if int(ix/50) == ix/50:
        
        try:
            df = pd.DataFrame(items)
            df.to_excel('jeans.xlsx')
            
            if len(errores) >0:
                
                pd.DataFrame(errores).to_excel('jeans_error.xlsx')
                
            file = open("jeans.txt", "w")
            file.write( str(url_chunk) + os.linesep)
            file.close()
        except:
            print("Erro al crear .xlsx")    
    
end_ = datetime.datetime.now()

In [16]:
# start_ = datetime.datetime.now()
# items = []

# batch_size = 5
# url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]

# var = 0
# for url_chunk in url_chunks:
#     items.append(scrape_batch(url_chunk))
#     var += 1
    
#     if var == 50:
#         var = 0
#         try:
#             df = pd.DataFrame(items)
#             df.to_excel('jeans.xlsx')
                                            
#             file = open("jeans.txt", "w")
#             file.write( str(url_chunk) + os.linesep)
#             file.close()
#         except:
#             print("Erro al crear .xlsx")
        
    
# end_ = datetime.datetime.now()

In [17]:
# ##COPIADOOO
# new_df = df[(df['tipo'] != 'JEAN') &
#             (df['tipo'] != 'CHAQUETA DE JEAN') &
#             (df['tipo'] != 'SHORT DE JEAN') &
#             (df['tipo'] != 'CALZA')]

# batch_size = 5
# url_chunks = [new_df[x:x+batch_size] for x in range(0, len(new_df), batch_size)]

# var = 0
# for url_chunk in url_chunks:
#     items.append(scrape_batch(url_chunk))
#     var += 1
    
#     if var == 50:
#         var = 0
#         try:
#             df = pd.DataFrame(items)
#             df.to_excel('jeans.xlsx')
                                            
#             file = open("jeans.txt", "w")
#             file.write( str(url_chunk) + os.linesep)
#             file.close()
#         except:
#             print("Erro al crear .xlsx")
        
    
# end_ = datetime.datetime.now()


In [18]:
print('Tiempo de ejecución: {}'.format(end_ - start_)[:-4])

Tiempo de ejecución: 2:12:22.23


In [19]:
new_list = aplanar_lista(items)
new_list = [i for i in new_list if type(i) != int]
dfShein = pd.DataFrame(new_list)

In [20]:
dfShein = dfShein.rename(columns={0: 'pos'})
dfShein = dfShein.rename(columns={1: 'Tipo'})
dfShein = dfShein.rename(columns={2: 'Color'})
dfShein = dfShein.rename(columns={3: 'id Producto'})
dfShein = dfShein.rename(columns={4: 'Descripcion'})
dfShein = dfShein.rename(columns={5: 'Talle'})
dfShein = dfShein.rename(columns={6: 'Stock'})
dfShein = dfShein.rename(columns={7: 'Precio'})
dfShein = dfShein.rename(columns={8: 'Precio_dto'})
dfShein = dfShein.rename(columns={9: 'Url Imagen'})
dfShein = dfShein.rename(columns={10: 'Url Producto'})
dfShein = dfShein.rename(columns={11: 'pagina_scraper'})

In [21]:
dfShein["Fecha"] = hoy
dfShein["Marca"] = "SHEIN"
dfShein["Moneda"] = "DOLAR US$"
dfShein['Sexo'] = 'Mujer'
dfShein['Descripcion aux'] = '0'

In [22]:
paginas = len(dfShein['pagina_scraper'].unique())

file = open("jeans.txt", "w")
file.write(  os.linesep +f'CANTIDAD DE ITEMS - {len(dfShein)}' ) 
file.write(  os.linesep +f'DURACION - {format(end_ - start_)[:-4]}' ) 
file.write(  os.linesep +f'CANTIDAD DE HOJAS - {paginas}' ) 
file.close()

In [23]:
if len(dfShein['Tipo'].unique().tolist()) != len(set([item[0] for item in Tlistas])):
    print('REVISAR LOS TIPOS SCRAPEADOS')
    raise

In [24]:
dfShein.to_excel(f'../Salida/dfShein_jeans{fecha}.xlsx')

In [25]:
len(dfShein.drop_duplicates()),len(dfShein),len(dfShein.dropna())

(19294, 19294, 19294)

In [26]:
# POR SI SE QUEDA CORRER ESTO
# startt_ = datetime.datetime.now()
# var = 0
# for url_chunk in copy_chunks:
#     items.append(scrape_batch(url_chunk))
#     var += 1
    
#     if var == 50:
#         var = 0
#         try:
#             df = pd.DataFrame(items)
#             df.to_excel('jeans.xlsx')
                                            
#             file = open("jeans.txt", "w")
#             file.write( str(url_chunk) + os.linesep)
#             file.close()
#         except:
#             print("Erro al crear .xlsx")
    
# end_ = datetime.datetime.now()